In [1]:
%matplotlib inline
import pandas as pd
#import matplotlib.pyplot as plt
from bqplot import pyplot as plt 

from __future__ import print_function
import numpy as np

from bqplot import (
    Axis, ColorAxis, LinearScale, DateScale, DateColorScale, OrdinalScale,
    OrdinalColorScale, ColorScale, Scatter, Lines, Figure, Tooltip
)

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

### Read the data.

<li> Read the data of the readings of the detectors in the dataframe data </li>
<li> Read the data of the location of the detectors in the dataframe locations </li>


In [2]:
## Read the csv file into the dataframe using pandas
data = pd.read_csv("data/data_tohoku_norm_transpose.csv", header=None)

locations = pd.read_csv("data/location.txt",
                        delimiter="\t", names =["longitude", "latitude", "a", "b"])
del locations["a"], locations["b"]

### Checking for the null values 

In [3]:
locations.isnull().sum()

longitude    0
latitude     0
dtype: int64

### Create an index for selecting the readings from the data based on time. 

Created a series using the daterange. 
The time has the same size as the no of rows in the detectors data.  

data.shape
(14401, 438)



In [4]:
time = pd.Series(range(0,data[0].size))

#### Creating a series of range for storing them as index in the location data frame. 

In [5]:
## number of station 
station = pd.Series(range(0,438))
station = station.astype(int)

### Intechanging the columns of the locations. 
The latitude is the first colum and the longitude is the last. 
Also adding a new column "station" and setting it as the index. 

In [6]:
cols = locations.columns.tolist()
cols = cols[-1:] + cols[:-1]
locations = locations[cols]

locations['station'] =station
locations.set_index('station',inplace=True)

locations.min()

latitude      26.463
longitude   -123.930
dtype: float64

from haversine import haversine

## lat and long values of the tohuku quake
tohuku_loc=[38.3,142.4]
tohuku_loc
x=[]


for i in station:
    #print(i)
    x.append(haversine(locations.loc[i], tohuku_loc,miles=True))
    #dist = haversine(locations.loc[i], tohuku_loc,miles=True)
    
locations['distance'] =x
location_sort=locations.sort_values('distance')

location_sort

data.min().min()
data.max().max()
data_norm = (data - data.min() ) / (data.max() - data.min())

### Creating index  in the data frame using the time.

In [7]:
data['time']=time
data.set_index('time',inplace=True)

In [8]:
data.loc[time == 0][0]

time
0   -0.000255
Name: 0, dtype: float64

### Normalizing the Data for displaying the detectors with the value of the reading based on the slider. 

In [9]:
data_norm = (data - data.min()) / (data.max() - data.min())
data_norm.loc[time==0][0]

time
0    0.453386
Name: 0, dtype: float64

### Creation of the slider using time

sc_x = LinearScale()
sc_y = LinearScale()
col_sc = ColorScale(min = data[0].min(), max =data[0].max())

import ipywidgets
#time_slide = interactive(interval = (time.min(), time.max(), 1))
def make_plot(interval = 0, station = 0 ):
    plt.subplot(211)

#map_mark = Map(map_data=topo_load('map_data/USStatesMap.json'), scales={'projection': AlbersUSA()})    
#scatt = Scatter(x=locations['latitude'], y=locations['longitude'], scales={'x': sc_x, 'y': sc_y,color})
plt.scatter(locations['latitude'], locations['longitude'], color=data_norm[time==10])
plt.show()

plt
#ax_x = Axis(scale=sc_x)
#ax_y = Axis(scale=sc_y)

#ax_c = ColorAxis(scale=col_sc)
#Figure(marks=[scatt], axes=[ax_x, ax_y,ax_c])
 



import cartopy as ccrs
import cartopy.io.shapereader as shpreader

plt.figure(figsize=(1280,720))
states=ccrs.io.shapereader.natural_earth(resolution='110m',category='cultural', name='admin_1_states_provinces_lakes_shp')
reader = ccrs.io.shapereader.Reader(states)
geometries = list(reader.geometries())
fig = plt.figure()

#ax= fig.add_subplot(111)
#plt.plot(data[0][0:3600])
#ax=fig.add_subplot(111, projection=ccrs.crs.LambertConformal())
ax= plt.axes(projection=ccrs.crs.Mercator())
ax.add_geometries(geometries,crs = ccrs.crs.PlateCarree(), facecolor=(0,1,0), linewidth=0.5, edgecolor='black')
ax.set_extent([-127,-74, 24, 47], ccrs.crs.Geodetic())

#ax=fig.add_subplot(211, projection =ccrs.crs.GOOGLE_MERCATOR)
plt.scatter(x=locations["longitude"],y=locations["latitude"], transform=ccrs.crs.Geodetic())

#fig.add_subplot(211)
#plt.scatter(x=locations["latitude"],y=locations["longitude"])

plt.show()



In [10]:
from IPython.display import Audio

In [11]:
normed = (data[0] - data[0].min())/(data[0].max() - data[0].min()) * 4 - 1
Audio(normed, rate=44100/8)

In [12]:
def wave_form_detect(station, time):
    """
    Get the data for the waveform from the data from for the selected station and time duration.
    """
    x = range(0, time)
    y = data[station][0:time]

    return x, y

def update_line(self, target):
    #print(sel_station)
    new_x, new_y = wave_form_detect(scat.selected, 14400)
    line.x = new_x
    line.y = new_y
    
def get_time(interval):
    
    return interval

def upd_col(time ): 
    temp = np.array(data_norm[time==time].values.flatten())
    return np.log(temp)

In [13]:
time_slide = interactive(get_time,interval=(time.min(), time.max(), 1))

In [14]:
#np.array(price_data[symbols[0]].values.flatten())


In [15]:

color = upd_col(time_slide.result)
color 

C:\Users\pratik\Anaconda3\lib\site-packages\ipykernel\__main__.py:22: RuntimeWarning: divide by zero encountered in log


array([-0.79938593, -0.79940708, -0.79942724, ..., -0.79276941,
       -0.79358106, -0.79437158])

In [ ]:
#time_slide.observe(upd_color,names=['time'])

In [ ]:
sc_geo = AlbersUSA()
states_map = Map(map_data=topo_load('map_data/USStatesMap.json'), scales={'projection': sc_geo})
Figure(marks=[states_map], title='US States Map Example')

In [ ]:
# Start by defining a scale for each axis
sc_x = LinearScale()

# The amplitudes are limited to ±1 for this example...
sc_y = LinearScale()

col_sc=ColorScale()

#color = np.log(color)

## Creatinig the scatter plot with the lat and long values of the detectores: 
## Included the "select" interaction with the click event. 

scat = Scatter(x=locations['latitude'] , y=locations['longitude'], scales={'x': sc_x, 'y': sc_y,'color': col_sc}, 
               #colors=['green'], scales={'x': sc_x, 'y': sc_y, 'color': sc_c1}
               #scale_type={'linear'},
               color=color ,
               stroke='black',
               # This is what makes this plot interactive
               interactions={'click': 'select'},
               selected_style={'opacity': 1.5, 'fill': 'blue', 'stroke': 'white'},
                       unselected_style={'opacity': 1.0})

# Only allow points to be moved vertically...
#scat.restrict_y = True

# Define the axes themselves
ax_x = Axis(scale=sc_x)
ax_y = Axis(scale=sc_y, tick_format='0.2f', orientation='vertical')


# The graph itself...
detector_loc = Figure(marks=[scat], axes=[ax_x, ax_y], 
                           title='Detector Location')

# This width is chosen just to make the plot fit nicely with 
# another. Change it if you want.
detector_loc.layout.width = '400px'

# Let's see what this looks like...
box = widgets.HBox(children=[detector_loc, time_slide])
#box

#type(color)

In [ ]:
initial_station = 0
initial_timeinterval=14400

In [ ]:
lin_x = LinearScale(min=0, max=14400)
lin_y = LinearScale()

# Note that here, unlike above, we do not set the initial data.
line = Lines(scales={'x': lin_x, 'y': lin_y}, colors=['red'],
               enable_move=False)

ax_x = Axis(scale=lin_x)
ax_y = Axis(scale=lin_y, tick_format='0.2f', orientation='vertical')

waveform = Figure(marks=[line], axes=[ax_x, ax_y], 
                title='Waveform Earthquake at Station:',
                # Honestly, I just like the way the animation looks.
                # Value is in milliseconds.
                animation_duration=500)

# Size as you wish...
waveform.layout.width = '400px'

# Calculate the waveform for the station for default values....
line.x, line.y = wave_form_detect(initial_station, initial_timeinterval)


#waveform



In [ ]:
# update line on change of x or y of scatter
#detector_loc.marks[0].observe(update_line,names=['y'])
#detector_loc.observe(update_line,names=['selected'])
#scat.on_element_click(update_line)

scat.on_element_click(update_line)

In [ ]:
from IPython.display import display
display(time_slide)
box = widgets.HBox(children=[detector_loc, waveform])
box